In [109]:
import numpy as nnp
import matplotlib.pyplot as plt

from lr_utils import *

In [110]:
# 고양이인 데이터와 고양이가 아닌 데이터를
# 로지스틱 회귀를 통해 구분해보기
train_set_x_orig, train_set_y, test_set_x_orig, test_set_y, classes = load_dataset()

In [111]:
train_set_x_flatten = train_set_x_orig.reshape(train_set_x_orig.shape[0],-1).T
test_set_x_flatten = test_set_x_orig.reshape(test_set_x_orig.shape[0],-1).T

In [112]:
#utils -param 시작, sigmoid

def initialize_with_zeros(dim):
    #zeros, ones는 zeros((차원1, 차원2))과 같은식으로 사용해야 함
    w = np.zeros((dim,1))
    b = 0.
    return w, b

def sigmoid(z):
    s = 1 / (1 + np.exp(-z))
    return s

In [113]:
def propagate(w ,b, X, Y):
    m = X.shape[1]
    Z = w.T@X + b
    A = sigmoid(Z)

    cost = (Y * np.log(A) + (1-Y) * np.log(1-A)).sum() / -m

    dw = (X @ (A-Y).T) / m
    db = (A-Y).sum() / m
    grads = {
        "dw" : dw,
        "db" : db,
    }
    return grads, cost

In [114]:
def optimize(w, b, X, Y, num_iterations = 2000, lr = 0.009, print_cost = True):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w,b,X,Y)
        dw = grads["dw"]
        db = grads["db"]
        #update param
        #deepcopy를 사용하지 않는다면 optimize 외부에서 넘어온 w와 b의 값이
        #어딘가에서 변할 수도 있음
        w = w- lr * dw
        b = b- lr * db
        
        if (i+1) % 100 ==0:
            costs.append(cost)

            if print_cost:
                print(f"Cost After {i+1} iterations : {cost}")
    params = {
        "w" : w, 
        "b" : b}
    
    grads = {"dw" : dw,
             "db" : db,}
    return params, grads, costs
    


In [115]:
train_set_x = train_set_x_flatten / train_set_x_flatten.max()
test_set_x = test_set_x_flatten / train_set_x_flatten.max() # 흠 스케일링을 똑같이 해야겠죠?
w,b = initialize_with_zeros(train_set_x.shape[0])
params, grads, costs = optimize(w,b,train_set_x,train_set_y, )

Cost After 100 iterations : 0.47715649504824453
Cost After 200 iterations : 0.8471092191546921
Cost After 300 iterations : 0.5837524449397078
Cost After 400 iterations : 0.48113380121374033
Cost After 500 iterations : 0.36121067514479355
Cost After 600 iterations : 0.24121803161715624
Cost After 700 iterations : 0.16856909780439225
Cost After 800 iterations : 0.15051961104077807
Cost After 900 iterations : 0.13960214924932912
Cost After 1000 iterations : 0.1303981662836805
Cost After 1100 iterations : 0.1223950112584566
Cost After 1200 iterations : 0.11532321915155781
Cost After 1300 iterations : 0.10901077034429985
Cost After 1400 iterations : 0.10333458691721242
Cost After 1500 iterations : 0.0982005870101832
Cost After 1600 iterations : 0.09353397980325093
Cost After 1700 iterations : 0.08927382998229712
Cost After 1800 iterations : 0.08536965873672563
Cost After 1900 iterations : 0.0817791391225479
Cost After 2000 iterations : 0.07846644198882445


In [116]:
def predict(w,b,X):
    m = X.shape[1]
    #y와 같은 차원으로 일단 배열을 만들어. 다 False인 상태
    Y_pred = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(w.T@X + b)
    #dataset의 갯수만큼
    for i in range(A.shape[1]):
        if A[0,i] > 0.5:
            Y_pred[0,i] = 1.
        else:
            Y_pred[0,i] = 0. #어차피 0으로 초기화되어있으니 안 써도 됨
    return Y_pred

In [117]:
# 이제 모델에 모두 통합해보기
# 모델의 input은train_x,y / test_x,y 
# 모델의 output은 여러 결과들일 거임
# 그 결과는 cost, prediction accuracy가 되겠죠?
# 경우에 따라서 w,b등의 파라미터도 output으로 나올 수 있을 것

In [118]:
'''
1. parameters initialize 하기
2. optimize를 통해서 params, grads, costs 산출하기
3. 산출한 params를 활용해서 predict하기
4. predict한 결과를 train에서와 test에서 비교해보며 분석해보기
'''
def model(X_train, Y_train, X_test, Y_test, num_iterations= 2000, lr= 0.09, print_cost = True, print_acc = True):
    '''
    dataset은 (-1, m) 형태로 flatten 되어있어야 하며
    사진과 같은 픽셀에서는 /255로 나눈 뒤에 집어넣어줘야 합니다.
    '''
    #prams 0으로 초기설정
    w, b = initialize_with_zeros(X_train.shape[0])
    params, grads, costs = optimize(w ,b ,X_train ,Y_train, num_iterations, lr,print_cost) 
    w = params.get("w")
    b = params.get("b")

    #w와 b가 확보되었으니, 확보된 w와 b를 활용해 train과 test에서 prediction해보기
    Y_pred_train = predict(w,b,X_train)
    Y_pred_test = predict(w,b,X_test)

    #print cost 옵션이 True라면 cost를 출력
    if print_acc:
        train_acc = (1 - np.mean(np.abs(Y_pred_train - Y_train))) * 100
        test_acc = (1 - np.mean(np.abs(Y_pred_test - Y_test))) * 100
        print(f"train_acc : {train_acc}%")
        print(f"test_acc : {test_acc}%")
    

    d = {"costs" : costs,
         "w" : w,
         "b" : b,
         "lr" : lr
         }
    return d


In [119]:
#자 이제 실제로 데이터를 모델에 넣는 시뮬레이션을 해봅시다!
#load dataset
X_train_orig, Y_train, X_test_orig, Y_test, classes = load_dataset() #class는 뭐하는건지 잘 모르겠습니다

In [120]:
#x와 y가 어떻게 생겼는지 개략적으로 파악
print(X_train_orig.shape)
print(Y_train.shape)

(209, 64, 64, 3)
(1, 209)


In [121]:
#x와 y가 구체적으로 어떻게 생겼는지 파악
display(X_train_orig)
display(Y_train)

array([[[[ 17,  31,  56],
         [ 22,  33,  59],
         [ 25,  35,  62],
         ...,
         [  1,  28,  57],
         [  1,  26,  56],
         [  1,  22,  51]],

        [[ 25,  36,  62],
         [ 28,  38,  64],
         [ 30,  40,  67],
         ...,
         [  1,  27,  56],
         [  1,  25,  55],
         [  2,  21,  51]],

        [[ 32,  40,  67],
         [ 34,  42,  69],
         [ 35,  42,  70],
         ...,
         [  1,  25,  55],
         [  0,  24,  54],
         [  1,  21,  51]],

        ...,

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],

        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
        

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [122]:
#model 요구사항에 맞춰서 데이터 전처리
X_train = X_train_orig.reshape(X_train_orig.shape[0], -1).T / 255.
X_test = X_test_orig.reshape(X_test_orig.shape[0], -1).T / 255.

In [123]:
logistic_reg_model = model(X_train, Y_train, X_test, Y_test)

C:\Users\eddie\AppData\Local\Temp\ipykernel_13296\2598877797.py:6: RuntimeWarning: divide by zero encountered in log
  cost = (Y * np.log(A) + (1-Y) * np.log(1-A)).sum() / -m
C:\Users\eddie\AppData\Local\Temp\ipykernel_13296\2598877797.py:6: RuntimeWarning: invalid value encountered in multiply
  cost = (Y * np.log(A) + (1-Y) * np.log(1-A)).sum() / -m


Cost After 100 iterations : 9.634550539428096
Cost After 200 iterations : 9.189480639570984
Cost After 300 iterations : 0.7335429872619103
Cost After 400 iterations : nan
Cost After 500 iterations : nan
Cost After 600 iterations : nan
Cost After 700 iterations : nan
Cost After 800 iterations : nan
Cost After 900 iterations : nan
Cost After 1000 iterations : nan
Cost After 1100 iterations : nan
Cost After 1200 iterations : nan
Cost After 1300 iterations : nan
Cost After 1400 iterations : nan
Cost After 1500 iterations : nan
Cost After 1600 iterations : nan
Cost After 1700 iterations : nan
Cost After 1800 iterations : nan
Cost After 1900 iterations : nan
Cost After 2000 iterations : nan
train_acc : 100.0%
test_acc : 65.99999999999999%
